## Projekt 2

Välkommen till projekt2. Cell -> Run all för att börja

Importerar **requests** för att kunna GET datan

Importerar **BeautifulSoup** för att kunna använda modulen för att göra datan lättare att arbeta med

Med **datetime** får jag in den nuvarande tiden och datumet

**regex(re)** Använder jag för att få ut endast den datan jag vill ha av ett element, t.ex procenten och antal reviews

In [11]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

**reqs** är en lista som sparar i sig alla sidor man GET i for loopen. 

Man kan öka antalet sidor om man vill, 5 by default eftersom det var önskat i uppgiftsbeskrivningen

In [12]:
reqs = []
#Hur många pages man vill ta
pageamount = 5
#Måste börja från 1 efter som https://store.steampowered.com/search/?specials=1&page=0 
##returnerar 50 spel vilket fick sida 1 att komma 2 ggr i datan
for x in range (1,pageamount+1):
    req = requests.get("https://store.steampowered.com/search/?specials=1&page="+str(x)).text
    reqs.append(req)

Gör sidorna till en soup mha **BeautifulSoup** För att göra data munging/scraping/wrangling lättare

Då man **parsar** datan blir det en **nested datastructure**

Sidorna som gjorts till soppa sparas i soup listan

In [13]:
soup = []
for num in range(0,len(reqs)):
    soup.append(BeautifulSoup(reqs[num],'html.parser')) 

Tar alla divs med rätt attributer och appendar dem till tables listan

In [14]:

attrs = {"id":"search_resultsRows"}
tables = []
for num in range(0,len(soup)):
    tables.append(soup[num].find("div",attrs))
    


I Cellen nedanför sker det mesta

**datadict** dicten skapar för varje iteration i loopen ett nytt spel

**datadict** appendas sedan till listan rows som kommer att innehålla alla spel

I loopen finns det checks för att t.ex se till att det inte blir error om ett spel har 0 reviews, kolla vilka operativ spelet kan spelas på.

RegEx används för att bryta ur väsentlig information av en divs text, t.ex antalet reviews

In [15]:
#rows kommer att innehålla alla spel
rows = []
#Loopen som loopar igenom alla spel på alla sidor
for num in range(len(tables)):
    for row in tables[num].findAll("div","responsive_search_name_combined"):
            #Spelen far in i datadict!
            datadict = {}
            datadict["game"]= row.find("span").get_text()
            
            #Ifall spelet är positivt rated
            if row.find("span",{"class":"search_review_summary positive"}):
                #digs bryter ut % och antalet reviews av tooltippen!
                digs = re.findall('[0-9]',row.find("span",{"class":"search_review_summary positive"})["data-tooltip-html"])
                #eftersom vi just nu får % och ratings allt i samma så bryter jag ut dom med pythonic slicing för att få ut 2 första siffrorna, alltså procenten
                #Dividerar med 7 och trar det * 100 för att konvertera till Steams rating skala
                #Om man skulle villa ha en skala 0-10 behöver man bara ändra /7 till /10!
                datadict["rating"] = str(int(digs[0:2][0]+digs[0:2][1])*7/100)
                #Den resterande dlen är antalet reviews
                datadict["reviews"] = ''.join(digs[2:])
                
                #Ifall spelet är mixed rated
            elif row.find("span",{"class":"search_review_summary mixed"}):
                #digs bryter ut % och antalet reviews av tooltippen!
                digs = re.findall('[0-9]',row.find("span",{"class":"search_review_summary mixed"})["data-tooltip-html"])
                datadict["rating"] = str(int(digs[0:2][0]+digs[0:2][1])*7/100)
                datadict["reviews"] = ''.join(digs[2:])
                print(digs[0:2])
                #Ifall spelet är negativt rated
            elif row.find("span",{"class":"search_review_summary negative"}):
                digs = re.findall('[0-9]',row.find("span",{"class":"search_review_summary negative"})["data-tooltip-html"])
                datadict["rating"] = str(int(digs[0:2][0]+digs[0:2][1])*7/100)
                datadict["reviews"] = ''.join(digs[2:])
            
            else:
                #Ifall det inte finns rating eller reviews 
                datadict["rating"] = None
                datadict["reviews"] = None
                #If satsen kollar att det inte finns tomma värden -> error handling
            if row.find("div",{"class":'col search_discount responsive_secondrow'}).span:
                datadict["discount"] = row.find("div",{"class":'col search_discount responsive_secondrow'}).span.get_text()
                datadict["newPrice"] = row.find("div",{"class":"col search_price discounted responsive_secondrow"}).get_text().split("€")[1]+"€"
                datadict["origPrice"] = row.find("div",{"class":"col search_price discounted responsive_secondrow"}).span.get_text()
            datadict["releaseYear"] = row.find("div",{"class":"col search_released responsive_secondrow"}).get_text()
           
                #Check ifall spelet är free to play. Endast f2p spel har "class":'col search_price responsive_secondrow'
                # eftersom de inte har en discount utan är gratis blir det search_price istället för search_discount
            if row.find("div",{"class":'col search_price responsive_secondrow'}):
                datadict["discount"] = "Free to Play"
                datadict["newPrice"] = "Free to Play"
                datadict["origPrice"] = "Free to Play"
                
                # Check av vilka operativ som stöds
            if row.find("span", attrs ={'class':'platform_img win'}):
                datadict["platformwin"] = "1" 
            else: 
                datadict["platformwin"] = "0"

            if row.find("span", attrs ={'class':'platform_img mac'}):
                datadict["platformmac"] = "1" 
            else: 
                datadict["platformmac"] = "0"

            if row.find("span", attrs ={'class':'platform_img linux'}):
                datadict["platformlinux"] = "1" 
            else: 
                datadict["platformlinux"] = "0"
                #Sätter in nuvarande tiden och datumet
            datadict["time"] = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
            #Slår in spelet och alla dess värden i rows!
            rows.append(datadict)
            


['5', '2']
['6', '5']
['6', '6']
['5', '0']
['6', '5']
['6', '4']
['4', '6']
['6', '4']


Steams review/rating system är utan att ta i beaktande antalet reviews en 7 point scale

dvs max rating är 7

In [16]:
import pandas as pd
#Gör om listan av alla spel och deras mungade värden till en DataFrame
df = pd.DataFrame(rows)
df


,game,rating,reviews,releaseYear,discount,newPrice,origPrice,platformwin,platformmac,platformlinux,time
0,Counter-Strike: Global Offensive,6.09,4996067,"21 Aug, 2012",Free to Play,Free to Play,Free to Play,1,1,1,2020-11-28-02:08:34
1,DOOM Eternal,6.16,79053,"19 Mar, 2020",-67%,"19,79€","59,99€",1,0,0,2020-11-28-02:08:34
2,Red Dead Redemption 2,5.67,119883,"5 Dec, 2019",-33%,"40,19€","59,99€",1,0,0,2020-11-28-02:08:34
3,Borderlands 3,5.81,45374,"13 Mar, 2020",-67%,"19,79€","59,99€",1,0,0,2020-11-28-02:08:34
4,The Sims™ 4,6.23,5690,"18 Jun, 2020",-88%,"4,79€","39,99€",1,0,0,2020-11-28-02:08:34
...,...,...,...,...,...,...,...,...,...,...,...
120,Wolfenstein: Youngblood,3.22,5810,"25 Jul, 2019",-70%,"8,99€","29,99€",1,0,0,2020-11-28-02:08:34
121,Elder Scrolls Summer Bundle,6.51,84961,,-70%,"22,25€","74,97€",1,0,0,2020-11-28-02:08:34
122,Gears 5,4.48,10051,"9 Sep, 2019",-75%,"9,99€","39,99€",1,0,0,2020-11-28-02:08:34
123,NEKOPARA Vol. 4,6.86,1523,"26 Nov, 2020",-10%,"7,37€","8,19€",1,0,0,2020-11-28-02:08:34


I cellena nedan får man en överblick av datan mha **tail()**, **head()** och **describe()**

In [17]:

df.head()

,game,rating,reviews,releaseYear,discount,newPrice,origPrice,platformwin,platformmac,platformlinux,time
0,Counter-Strike: Global Offensive,6.09,4996067,"21 Aug, 2012",Free to Play,Free to Play,Free to Play,1,1,1,2020-11-28-02:08:34
1,DOOM Eternal,6.16,79053,"19 Mar, 2020",-67%,"19,79€","59,99€",1,0,0,2020-11-28-02:08:34
2,Red Dead Redemption 2,5.67,119883,"5 Dec, 2019",-33%,"40,19€","59,99€",1,0,0,2020-11-28-02:08:34
3,Borderlands 3,5.81,45374,"13 Mar, 2020",-67%,"19,79€","59,99€",1,0,0,2020-11-28-02:08:34
4,The Sims™ 4,6.23,5690,"18 Jun, 2020",-88%,"4,79€","39,99€",1,0,0,2020-11-28-02:08:34


In [18]:
df.tail()

,game,rating,reviews,releaseYear,discount,newPrice,origPrice,platformwin,platformmac,platformlinux,time
120,Wolfenstein: Youngblood,3.22,5810,"25 Jul, 2019",-70%,"8,99€","29,99€",1,0,0,2020-11-28-02:08:34
121,Elder Scrolls Summer Bundle,6.51,84961,,-70%,"22,25€","74,97€",1,0,0,2020-11-28-02:08:34
122,Gears 5,4.48,10051,"9 Sep, 2019",-75%,"9,99€","39,99€",1,0,0,2020-11-28-02:08:34
123,NEKOPARA Vol. 4,6.86,1523,"26 Nov, 2020",-10%,"7,37€","8,19€",1,0,0,2020-11-28-02:08:34
124,Pummel Party,6.37,16780,"20 Sep, 2018",-40%,"7,49€","12,49€",1,0,0,2020-11-28-02:08:34


In [19]:
df.describe()

,game,rating,reviews,releaseYear,discount,newPrice,origPrice,platformwin,platformmac,platformlinux,time
count,125,124,124,125,125,125,125,125,125,125,125
unique,124,33,123,108,24,57,29,1,2,2,1
top,Shadow of the Tomb Raider: Definitive Edition,6.51,52320,,-75%,"14,99€","39,99€",1,0,0,2020-11-28-02:08:34
freq,2,8,2,8,17,12,25,125,95,99,125


Nedanför sköts sparningen av datan in i en CSV fil

In [20]:
import os
#Om filen inte finns
if not os.path.isfile('steamsaledata.csv'):
   df.to_csv('steamsaledata.csv', header='column_names')
#Om filen finns
else:
   df.to_csv('steamsaledata.csv', mode='a', header=False)